In [1]:
import re
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import *
from deep_translator import GoogleTranslator
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')
path = '../../../datasets/garanti-bbva-data-camp/clean_education_v2.csv'

def my_tokenizer(text):
    return re.split("\\s+",text)

In [2]:
#df= pd.read_csv(path)
#print(df['user_id'].nunique())
#df.head()

In [3]:
def load_school(path: str, size: int = 20, exact_match: bool = True) -> pd.DataFrame:

    df_ = pd.read_csv(path)[['user_id', 'school_name']]

    if exact_match:
        most_freq_schools = df_["school_name"].value_counts()[:size].keys().tolist()
        for school in tqdm(most_freq_schools):
            df_[f"school_name_{school}"] = df_["school_name"].apply(lambda x: 1 if school == x else 0)

        return (
            df_.drop(columns=["school_name"], axis=1)
            .groupby(by=["user_id"], as_index=False)
            .sum().merge(
                df_.groupby(by="user_id", as_index=False).agg(
                    total_education=("school_name", "count")
                ),
                on=["user_id"],
                how="left",
            )
        )
    
    else:
        vectorizer = CountVectorizer(
            max_features=size,
            stop_words=stopwords.words("english"),
            ngram_range=(1, 3),
        )

        return (
            pd.DataFrame(
                vectorizer.fit_transform(df_["school_name"]).toarray(),
                columns=[f"school_name_{str(f)}" for f in vectorizer.get_feature_names()],
            )
            .assign(user_id=df_["user_id"].tolist())
            .groupby(by="user_id", as_index=False)
            .sum()
            .merge(
                df_.groupby(by="user_id", as_index=False).agg(
                    total_education=("school_name", "count")
                ),
                on=["user_id"],
                how="left",
            )
        )
        

In [4]:
def load_degree(path: str, size: int = 20, exact_match: bool = True) -> pd.DataFrame:

    df_ = pd.read_csv(path)[['user_id', 'degree']].fillna('')

    if exact_match:
        most_freq_degrees = df_.loc[df_['degree'] != '', 'degree'].value_counts()[:size].keys().tolist()
        for degree in tqdm(most_freq_degrees):
            df_[f"degree_{degree}"] = df_["degree"].apply(lambda x: 1 if degree == x else 0)

        return (
            df_.drop(columns=["degree"], axis=1)
            .groupby(by=["user_id"], as_index=False)
            .sum()
        )
    
    else:
        vectorizer = CountVectorizer(
            max_features=size,
            stop_words=stopwords.words("english"),
            ngram_range=(1, 2),
        )

        return (
            pd.DataFrame(
                vectorizer.fit_transform(df_["degree"]).toarray(),
                columns=[f"degree_{str(f)}" for f in vectorizer.get_feature_names()],
            )
            .assign(user_id=df_["user_id"].tolist())
            .groupby(by="user_id", as_index=False)
            .sum()
        )
        

In [5]:
def load_study(path: str, size: int = 20, exact_match: bool = True) -> pd.DataFrame:

    df_ = pd.read_csv(path)[['user_id', 'fields_of_study']].fillna('')

    if exact_match:
        most_freq_studies = df_.loc[df_['fields_of_study'] != '', 'fields_of_study'].value_counts()[:size].keys().tolist()
        for study in tqdm(most_freq_studies):
            df_[f"fields_of_study_{study}"] = df_["fields_of_study"].apply(lambda x: 1 if study == x else 0)

        return (
            df_.drop(columns=["fields_of_study"], axis=1)
            .groupby(by=["user_id"], as_index=False)
            .sum()
        )
    
    else:
        vectorizer = CountVectorizer(
            max_features=size,
            stop_words=stopwords.words("english"),
            ngram_range=(1, 3),
        )

        return (
            pd.DataFrame(
                vectorizer.fit_transform(df_["fields_of_study"]).toarray(),
                columns=[f"fields_of_study_{str(f)}" for f in vectorizer.get_feature_names()],
            )
            .assign(user_id=df_["user_id"].tolist())
            .groupby(by="user_id", as_index=False)
            .sum()
        )
        

In [48]:
#degree_df = load_degree(path, exact_match=False)
#
#print(degree_df['user_id'].nunique())
#
#degree_df.head()

In [17]:
#school_df = load_study(path, size = 55, exact_match=True)
#
#print(school_df['user_id'].nunique())
#
#school_df.head()

In [16]:
#[col for col in school_df.columns if ',' in col]

In [50]:
#df = school_df.merge(degree_df, on =['user_id'], how = 'left')

In [34]:
#school_grouped = df[['user_id']].drop_duplicates().merge(df.dropna(subset = ['school_name']).groupby(by='user_id', as_index=False).agg({'school_name': lambda #x: [s.strip() for s in ", ".join(x.unique()).split(",")]}), on = ['user_id'], how ='left').assign(school_name = lambda x: x.school_name.fillna("").apply(list))

In [62]:
#grouped = df[['user_id']].drop_duplicates().merge(df.dropna(subset = ['fields_of_study']).groupby(by='user_id', as_index=False).agg({'fields_of_study': lambda #x: [s.strip() for s in ", ".join(x.unique()).split(",")]}), on = ['user_id'], how ='left').assign(fields_of_study = lambda x: x.fields_of_study.fillna("").apply#(list))
#
#grouped['total_studies'] = grouped['fields_of_study'].apply(lambda x: len(x))
#
#grouped

In [33]:
#def vectorize(df: pd.DataFrame, grouped_df: pd.DataFrame, vectorizer, feature: str) -> pd.DataFrame:
#
#    comma_df = pd.DataFrame()
#    users_using_comma = df.loc[df[feature].astype(str).str.contains(","), "user_id"].tolist()
#
#    for i, j in zip(
#        grouped_df.loc[grouped_df["user_id"].isin(users_using_comma), "user_id"],
#        grouped_df.loc[grouped_df["user_id"].isin(users_using_comma), feature],):
#        for s in j:
#            comma_df = comma_df.append(
#                pd.DataFrame({"user_id": [i], feature: [s]})
#            )
#            
#    df_to_vectorize = (df.loc[~df[feature].astype(str).str.contains(",")].append(comma_df.drop_duplicates()).fillna({feature: ""})).copy()
#
#    return pd.DataFrame(
#            vectorizer.fit_transform(df_to_vectorize[feature]).toarray(),
#            columns=[f"{feature}_{str(f)}" for f in vectorizer.get_feature_names()],
#        ).assign(user_id=df_to_vectorize["user_id"].tolist()).groupby(by="user_id", as_index=False).sum()

In [34]:
#def load_education(
#    path: str, study_size: int = 10, degree_size: int = 10, school_size: int = 10, exact_match: bool = True
#) -> pd.DataFrame:
#
#    df_ = pd.read_csv(path)
#
#    school_grouped = df_[['user_id']].drop_duplicates().merge(df_.dropna(subset = ['school_name']).groupby(by='user_id', as_index=False).agg({'school_name': #lambda x: [s.strip() for s in ", ".join(x.unique()).split(",")]}), on = ['user_id'], how ='left').assign(school_name = lambda x: x.school_name.fillna("").#apply(list))
#
#    degree_grouped = df_[['user_id']].drop_duplicates().merge(df_.dropna(subset = ['degree']).groupby(by='user_id', as_index=False).agg({'degree': lambda x: [s.#strip() for s in ", ".join(x.unique()).split(",")]}), on = ['user_id'], how ='left').assign(degree = lambda x: x.degree.fillna("").apply(list))
#
#    study_grouped = df_[['user_id']].drop_duplicates().merge(df_.dropna(subset = ['fields_of_study']).groupby(by='user_id', as_index=False).agg#({'fields_of_study': lambda x: [s.strip() for s in ", ".join(x.unique()).split(",")]}), on = ['user_id'], how ='left').assign(fields_of_study = lambda x: x.#fields_of_study.fillna("").apply(list))
#
#    school_grouped['total_schools'] = school_grouped['school_name'].apply(lambda x: len(x))
#    degree_grouped['total_degrees'] = degree_grouped['degree'].apply(lambda x: len(x))
#    study_grouped['total_studies'] = study_grouped['fields_of_study'].apply(lambda x: len(x))
#
#    print(f'school_grouped shape: {school_grouped.shape}')
#    print(f'degree_grouped shape: {degree_grouped.shape}')
#    print(f'study_grouped shape: {study_grouped.shape}')
#    
#    if exact_match:
#
#        most_freq_school_names = (
#            df_["school_name"].value_counts()[:school_size].keys().tolist()
#        )
#        most_freq_studies = (
#            df_["fields_of_study"].value_counts()[:study_size].keys().tolist()
#        )
#        most_freq_degrees = df_["degree"].value_counts()[:degree_size].keys().tolist()
#
#        for school in tqdm(most_freq_school_names):
#            school_grouped[f'school_{school}'] = school_grouped['school_name'].apply(lambda x: 1 if school in x else 0)
#
#        for degree in tqdm(most_freq_degrees):
#            degree_grouped[f'degree_{degree}'] = degree_grouped['degree'].apply(lambda x: 1 if degree in x else 0)
#
#        for study in tqdm(most_freq_studies):
#            study_grouped[f'study_{study}'] = study_grouped['fields_of_study'].apply(lambda x: 1 if study in x else 0)
#
#        school_grouped = school_grouped.drop(columns = ['school_name'], axis = 1)
#        degree_grouped = degree_grouped.drop(columns = ['degree'], axis = 1)
#        study_grouped = study_grouped.drop(columns = ['fields_of_study'], axis = 1)
#
#        education = school_grouped.merge(degree_grouped, on = ['user_id'], how = 'left')
#        education = education.merge(study_grouped, on = ['user_id'], how = 'left')
#
#        return education
#
#    else:
#
#        school_df = df_[['user_id', 'school_name']].copy()
#        degree_df = df_[['user_id', 'degree']].copy()
#        study_df = df_[['user_id', 'fields_of_study']].copy()
#
#        school_vectorizer = CountVectorizer(
#            max_features=school_size,
#            stop_words=stopwords.words("english"),
#            ngram_range=(1, 3),
#            tokenizer=my_tokenizer
#            )
#
#        degree_vectorizer = CountVectorizer(
#            max_features=degree_size,
#            stop_words=stopwords.words("english"),
#            ngram_range=(1, 2),
#            #tokenizer=my_tokenizer
#            )
#
#        study_vectorizer = CountVectorizer(
#            max_features=study_size,
#            stop_words=stopwords.words("english"),
#            ngram_range=(1, 2),
#            #tokenizer=my_tokenizer
#            )
#
#        school_vectorized = vectorize(school_df, school_grouped, school_vectorizer, 'school_name').merge(school_grouped[['user_id', 'total_schools']], on = #'user_id', how = 'left')
#        degree_vectorized = vectorize(degree_df, degree_grouped, degree_vectorizer, 'degree').merge(degree_grouped[['user_id', 'total_degrees']], on = #'user_id', how = 'left')
#        study_vectorized = vectorize(study_df, study_grouped, study_vectorizer, 'fields_of_study').merge(study_grouped[['user_id', 'total_studies']], on = #'user_id', how = 'left')
#
#        education = school_vectorized.merge(degree_vectorized, on = ['user_id'], how = 'left')
#        education = education.merge(study_vectorized, on = ['user_id'], how = 'left')
#
#        return education

In [ ]:
#def load_school(path: str, size: int, exact_match: bool = True) -> pd.DataFrame:
#
#    df_ = pd.read_csv(path)
#
#    school_grouped = df_[['user_id']].drop_duplicates().merge(df_.dropna(subset = ['school_name']).groupby(by='user_id', as_index=False).agg({'school_name': #lambda x: [s.strip() for s in ", ".join(x.unique()).split(",")]}), on = ['user_id'], how ='left').assign(school_name = lambda x: x.school_name.fillna("").#apply(list))
#
#    school_grouped['total_schools'] = school_grouped['school_name'].apply(lambda x: len(x))
#    print(f'school_grouped shape: {school_grouped.shape}')
#
#    if exact_match:
#
#        most_freq_school_names = (
#            df_["school_name"].value_counts()[:size].keys().tolist()
#        )
#
#        #school_grouped['school_name'] = school_grouped['school_name'].apply(lambda x: ' '.join(x))
#        for school in tqdm(most_freq_school_names):
#            school_grouped[f'school_{school}'] = school_grouped['school_name'].apply(lambda x: 1 if school in x else 0)
#
#        return school_grouped.drop(columns = ['school_name'], axis = 1)
#    
#    else:
#
#        school_df = df_[['user_id', 'school_name']].copy()
#        school_vectorizer = CountVectorizer(
#            max_features=size,
#            stop_words=stopwords.words("english"),
#            ngram_range=(1, 3),
#            )
#
#        school_vectorized = vectorize(school_df, school_grouped, school_vectorizer, 'school_name').merge(school_grouped[['user_id', 'total_schools']], on = #'user_id', how = 'left')
#
#        return school_vectorized
#        

In [ ]:
#def load_degree(path: str, size: int, exact_match: bool = True) -> pd.DataFrame:
#
#    df_ = pd.read_csv(path)
#
#    degree_grouped = df_[['user_id']].drop_duplicates().merge(df_.dropna(subset = ['degree']).groupby(by='user_id', as_index=False).agg({'degree': lambda x: [s.#strip() for s in ", ".join(x.unique()).split(",")]}), on = ['user_id'], how ='left').assign(degree = lambda x: x.degree.fillna("").apply(list))
#
#    degree_grouped['total_degrees'] = degree_grouped['degree'].apply(lambda x: len(x))
#    print(f'degree_grouped shape: {degree_grouped.shape}')
#
#    if exact_match:
#
#        most_freq_degrees = (
#            df_["degree"].value_counts()[:size].keys().tolist()
#        )
#
#        #degree_grouped['degree'] = degree_grouped['degree'].apply(lambda x: ' '.join(x))
#        for degree in tqdm(most_freq_degrees):
#            degree_grouped[f'degree_{degree}'] = degree_grouped['degree'].apply(lambda x: 1 if degree in x else 0)
#
#        return degree_grouped.drop(columns = ['degree'], axis = 1)
#    
#    else:
#
#        degree_df = df_[['user_id', 'degree']].copy()
#        degree_vectorizer = CountVectorizer(
#            max_features=size,
#            stop_words=stopwords.words("english"),
#            ngram_range=(1, 2),
#            )
#
#        degree_vectorized = vectorize(degree_df, degree_grouped, degree_vectorizer, 'degree').merge(degree_grouped[['user_id', 'total_degrees']], on = #'user_id', how = 'left')
#
#        return degree_vectorized
#        

In [ ]:
#def load_study(path: str, size: int, exact_match: bool = True) -> pd.DataFrame:
#
#    df_ = pd.read_csv(path)
#
#    study_grouped = df_[['user_id']].drop_duplicates().merge(df_.dropna(subset = ['fields_of_study']).groupby(by='user_id', as_index=False).agg#({'fields_of_study': lambda x: [s.strip() for s in ", ".join(x.unique()).split(",")]}), on = ['user_id'], how ='left').assign(fields_of_study = lambda x: x.#fields_of_study.fillna("").apply(list))
#
#    study_grouped['total_studies'] = study_grouped['fields_of_study'].apply(lambda x: len(x))
#    print(f'study_grouped shape: {study_grouped.shape}')
#
#    if exact_match:
#
#        most_freq_studies = (
#            df_["fields_of_study"].value_counts()[:size].keys().tolist()
#        )
#
#        #study_grouped['fields_of_study'] = study_grouped['fields_of_study'].apply(lambda x: ' '.join(x))
#        for study in tqdm(most_freq_studies):
#            study_grouped[f'fields_of_study_{study}'] = study_grouped['fields_of_study'].apply(lambda x: 1 if study in x else 0)
#
#        return study_grouped.drop(columns = ['fields_of_study'], axis = 1)
#    
#    else:
#
#        study_df = df_[['user_id', 'fields_of_study']].copy()
#
#        study_vectorizer = CountVectorizer(
#            max_features=size,
#            stop_words=stopwords.words("english"),
#            ngram_range=(1, 3),
#            )
#
#        study_vectorized = vectorize(study_df, study_grouped, study_vectorizer, 'fields_of_study').merge(study_grouped[['user_id', 'total_studies']], on = #'user_id', how = 'left')
#
#        return study_vectorized
        

In [2]:
def load_education_v2(path: str, study_size: int = 10, degree_size: int = 10, school_size: int = 10) -> pd.DataFrame:

    df_ = pd.read_csv(path)
    df_ = df_.drop(columns = ['start_year_month', 'end_year_month'], axis = 1)

    ################################################################################################################

    df_ = fix_studies(df_)

    ################################################################################################################

    df_ = fix_school_names(df_)

    ################################################################################################################

    df_ = fix_degree(df_)

    #df_ = df_.drop_duplicates()
    df_['fields_of_study'] = df_['fields_of_study'].apply(lambda x: str(x).lower().strip())
    df_['fields_of_study'] = df_['fields_of_study'].apply(lambda x: translation(str(x)))
    for col in ['degree', 'fields_of_study', 'school_name']:
        df_[col] = df_[col].apply(lambda x: str(x).lower().strip())
        df_[col] = df_[col].apply(lambda x: str(x).replace('universitesi', 'university'))
        df_[col] = df_[col].apply(lambda x: str(x).replace('üniversitesi', 'university'))
        df_[col] = df_[col].apply(lambda x: str(x).replace('lisesi', 'high school'))
        df_[col] = df_[col].apply(lambda x: str(x).replace('ilköğretim okulu', 'primary school'))
        df_[col] = df_[col].apply(lambda x: translation(str(x)))
        df_[col] = df_[col].replace('nan', '')


    ################################################################################################################

    #study_df = df_.groupby(by = 'user_id', as_index = False).agg(total_studies = ('fields_of_study', 'nunique'))
    study_grouped = df_.groupby(by='user_id', as_index=False).agg({'fields_of_study': lambda x: ' '.join(x.unique())})

    vectorizer = CountVectorizer(max_features=study_size,
                                 stop_words=stopwords.words("english"),
                                 ngram_range=(1,2))

    study_grouped = pd.DataFrame(
        vectorizer.fit_transform(study_grouped["fields_of_study"]).toarray(),
        columns=[f'study_{str(f)}' for f in vectorizer.get_feature_names()],
    ).assign(user_id = study_grouped['user_id'])

    ################################################################################################################

    #degree_df = df_.groupby(by = 'user_id', as_index = False).agg(total_degrees = ('degree', 'nunique'))
    degree_grouped = df_.groupby(by='user_id', as_index=False).agg({'degree': lambda x: ' '.join(x.unique())})

    vectorizer = CountVectorizer(max_features=degree_size,
                                 stop_words=stopwords.words("english"),
                                 ngram_range=(1,2))
    
    degree_grouped = pd.DataFrame(
        vectorizer.fit_transform(degree_grouped["degree"]).toarray(),
        columns=[f'degree_{str(f)}' for f in vectorizer.get_feature_names()],
    ).assign(user_id = degree_grouped['user_id'])

    ################################################################################################################

    #school_df = df_.groupby(by = 'user_id', as_index = False).agg(total_school = ('school_name', 'nunique'))
    most_freq_school_names = df_['school_name'].value_counts()[:school_size].keys().tolist()
    school_grouped = df_.groupby(by='user_id', as_index=False).agg({'school_name': lambda x: ' '.join(x.unique())})
#
    #vectorizer = CountVectorizer(max_features=study_size,
    #                             stop_words=stopwords.words("english"),
    #                             ngram_range=(2,3))
    #
    #school_grouped = pd.DataFrame(
    #    vectorizer.fit_transform(school_grouped["school_name"]).toarray(),
    #    columns=[f'school_{str(f)}' for f in vectorizer.get_feature_names()],
    #).assign(user_id = school_grouped['user_id'])
    for school in tqdm(most_freq_school_names):
        school_grouped[f'school_{school}'] = school_grouped['school_name'].apply(lambda x: 1 if school in x else 0)
    school_grouped = school_grouped.drop(columns = ['school_name'], axis = 1)

#
    #for study in tqdm(most_freq_studies):
    #    study_grouped[f'study_{study}'] = study_grouped['fields_of_study'].apply(lambda x: 1 if study in x else 0)
#
    #for degree in tqdm(most_freq_degrees):
    #    degree_grouped[f'degree_{degree}'] = degree_grouped['degree'].apply(lambda x: 1 if degree in x else 0)
#
    #study_grouped = study_grouped.drop(columns=['fields_of_study'], axis=1)
#
    #degree_grouped = degree_grouped.drop(columns=['degree'], axis=1)

    grouped = study_grouped.merge(degree_grouped, on = ['user_id'], how = 'left')
    grouped = grouped.merge(school_grouped, on = ['user_id'], how = 'left')
    #grouped = grouped.merge(study_df, on = ['user_id'], how = 'left')
    #grouped = grouped.merge(degree_df, on = ['user_id'], how = 'left')
    #grouped = grouped.merge(school_df, on = ['user_id'], how = 'left')

    #grouped = grouped.merge(df_.groupby(by='user_id', as_index=False).agg(
    #    school_count=('school_name', 'count')), on=['user_id'], how='left')
    #grouped = grouped.merge(df_.groupby(by='user_id', as_index=False).agg(
    #    study_count=('fields_of_study', 'count')), on=['user_id'], how='left')
    #grouped = grouped.merge(df_.groupby(by='user_id', as_index=False).agg(
    #    school_nunique=('school_name', 'nunique')), on=['user_id'], how='left')
    #grouped = grouped.merge(df_.groupby(by='user_id', as_index=False).agg(
    #    study_nunique=('fields_of_study', 'nunique')), on=['user_id'], how='left')

    return grouped

In [3]:
#df = load_education(path)
#print(df.shape)
#df.head()

In [4]:
#translated = dict()
#
#for i in tqdm(df['school_name'].unique()):
#    translated[i] = GoogleTranslator(source='auto', target='en').translate(i)

In [5]:
#for i in df.loc[df['school_name'].astype(str).str.contains('Lisesi'), 'school_name'].value_counts()[:50].keys():
#    translated = GoogleTranslator(source='auto', target='en').translate(i)
#    if df.loc[df['school_name'] == translated].shape[0] != 0:
#        print(f'df.loc[df["school_name"] == "{i}", "school_name"] = "{translated.title()}"')

df.loc[df["school_name"] == "Sekine Evren Anadolu Lisesi", "school_name"] = "Sekine Evren Anatolian High School"
df.loc[df["school_name"] == "Zeytinburnu Teknik Lisesi", "school_name"] = "Zeytinburnu Technical High School"
df.loc[df["school_name"] == "İstanbul Ticaret Odası Anadolu Teknik Lisesi", "school_name"] = "Istanbul Chamber Of Commerce Anatolian Technical High School"
df.loc[df["school_name"] == "Üsküdar Anadolu Lisesi", "school_name"] = "Uskudar Anatolian High School"
df.loc[df["school_name"] == "Aydın Fen Lisesi", "school_name"] = "Aydın Science High School"


In [151]:
#df.loc[df['school_name'].str.contains('Sivas'), 'school_name'].value_counts()[:20].keys().tolist()

In [152]:
#df.loc[df['school_name'].str.contains('Üniversitesi'), 'school_name'].value_counts()

In [111]:
#df.loc[df['school_name'].str.contains('University'), 'school_name'].value_counts()[:20]